In [1]:
%load_ext autoreload
%autoreload 2

import os
import matplotlib.pyplot as plt

# Change working directory to project root
os.chdir('../../')

import torch
import numpy as np
from pathlib import Path
from tqdm import tqdm
import json
import imageio
from hmr4d.utils.wis3d_utils import make_wis3d
from hmr4d.utils.wis3d_utils import add_joints22_motion_as_lines
from hmr4d.dataset.rich.rich_utils import get_w2az_sahmr, parse_seqname_info
from hmr4d.utils.geo_transform import apply_T_on_points

### Random Noise

In [ ]:
import torch
motion_ayf = torch.zeros((120, 22, 3))  # dummy motion

idx = 0
gmpjpe, mpjpe = [], []
for idx in range(512):
    torch_generator = torch.Generator().manual_seed(idx)
    cond_motion = 0.02 * torch.randn(motion_ayf.shape, generator=torch_generator) + motion_ayf.clone()
    cond_motion = 0.15 * torch.randn(cond_motion.shape[0], generator=torch_generator)[:, None, None] + cond_motion

    gmpjpe.append((cond_motion - motion_ayf).norm(dim=-1, p=2).mean(dim=-1).mean() * 1000)
    mpjpe.append(((cond_motion - cond_motion[:, [0]]) - (motion_ayf - motion_ayf[:, [0]])).norm(dim=-1, p=2).mean()  * 1000)

gmpjpe = torch.stack(gmpjpe)
mpjpe = torch.stack(mpjpe)
gmpjpe_cummean = gmpjpe.cumsum(dim=0) / torch.arange(1, gmpjpe.shape[0] + 1)
mpjpe_cummean = mpjpe.cumsum(dim=0) / torch.arange(1, mpjpe.shape[0] + 1)
# print(f'gmpjpe: {gmpjpe:.4f}, mpjpe: {mpjpe:.4f}')

In [ ]:
gmpjpe_cummean = gmpjpe.cumsum(dim=0) / torch.arange(1, gmpjpe.shape[0] + 1)
mpjpe_cummean = mpjpe.cumsum(dim=0) / torch.arange(1, mpjpe.shape[0] + 1)
# Change to subplots
fig, axs = plt.subplots(2, 1, figsize=(10, 8))
axs[0].plot(gmpjpe_cummean, label='gmpjpe')
axs[1].plot(mpjpe_cummean, label='mpjpe')
# Add titles
axs[0].set_title('Global MPJPE')
axs[1].set_title('MPJPE')

plt.show()

### inspect image clip

#### dataset (manual)

In [5]:
from hmr4d.dataset.rich.rich_motion import Dataset
args = {
    "split": "val",
    "img_fps": 1,  # TODO: this is semantically wrong, should be "img_interval"
}
dataset = Dataset(**args) 


In [6]:
for idx in range(1010, 1014):
    data = dataset[idx]
    images = (data['prompt_imgs'] * 255).numpy().astype(np.uint8).transpose(0, 2, 3, 1)
    out_fn = f"tmp_{idx}.mp4"
    imageio.mimwrite(out_fn, images, fps=4)

#### dataset (loop to see how img_interval affect the resulting video)

In [7]:
from hmr4d.dataset.rich.rich_motion import Dataset

out_folder = Path("check_invertal")
out_folder.mkdir(exist_ok=True)
for interval in range(1, 30):
    args = {"split":"val", "img_fps": interval}
    dataset = Dataset(**args)

    idx= 1011
    data = dataset[idx]
    images = (data['prompt_imgs'] * 255).numpy().astype(np.uint8).transpose(0, 2, 3, 1)
    out_fn = out_folder / f"{idx}_{interval}.mp4"
    imageio.mimwrite(out_fn, images, fps=30/interval)



#### raw-code

In [ ]:
from hmr4d.dataset.rich.rich_utils import get_seqnames_of_split, get_img_fn
split = "val"
seqnames_split = get_seqnames_of_split(split)
img_root = Path(f"inputs/RICH/images_ds4/{split}")

In [ ]:
# from PIL import Image
import imageio

seqname = seqnames_split[10]
cam_id = 0
# Read images and dump as a video
imgs = []
sample_frames = 32
for f_id in range(50, 170, int(120 / sample_frames)):
    img_fn = get_img_fn(img_root, seqname, cam_id, f_id)
    imgs.append(imageio.imread(img_fn))

# Save as mp4 using imageio, with best quality
out_fn = f"tmp_{sample_frames}.mp4"
fps = sample_frames / 4
imageio.mimsave(out_fn, imgs, fps=fps, quality=5)  # default=5

### data-range (motion and image)

In [ ]:
rich_joints3d = np.load("inputs/RICH/hmr4d_support/joints3d/joints3d_smpl_v2.npy", allow_pickle=True).item()
seqname2imgrange = json.load(open("hmr4d/dataset/rich/resource/seqname2imgrange.json", "r"))

# CHECK total length is equal
for seqname in rich_joints3d:
    joints3d = rich_joints3d[seqname][0]
    total_length_motion = joints3d.shape[0]

    total_length_img = seqname2imgrange[seqname][1] - seqname2imgrange[seqname][0] + 1
    if total_length_motion != total_length_img:
        print(seqname, total_length_motion, total_length_img)